In [593]:
import numpy as np
import math
from scipy.spatial import distance
from scipy.optimize import curve_fit
from copy import copy
from random import random
from tqdm import tqdm

In [594]:
import sys

In [595]:
EPS = 1e-6

def curve(x, a, b):
    return 1.0 / (1.0 + a * x ** (2 * b))
    

def force_curve(x, w, a, b):
    x2 = np.sum(x ** 2)
    return (-2 * a * b * x2 ** (b - 1)) / (1 + x2) * w * x
    

def repilsive_curve(x, w, a, b):
    x2 = np.sum(x ** 2)
    return (2 * b) / ((EPS + x2) * (1 + a * x2 ** b)) * (1 - w) * x
    
    
def find_ab_params(min_dist):
    xv = np.linspace(0.0, 1.0, 300)
    yv = np.zeros(xv.shape)
    yv[xv < min_dist] = 1.0
    yv[xv >= min_dist] = np.exp(-(xv[xv >= min_dist] - min_dist))
    params, covar = curve_fit(curve, xv, yv)
    return params[0], params[1]


class Umap(object):
    
    def __init__(self, n_neighbors=5, n_targets=2, min_dist=0.0, n_epochs=100):
        self.n_neighbors = n_neighbors
        self.n_targets = n_targets
        self.n_epochs = n_epochs
        self.a, self.b = find_ab_params(min_dist)
    
    def fit_transform(self, X):
        fs_set = np.array([np.zeros(len(X)) for i in range(len(X))])
        for x_i in tqdm(range(len(X))):
            self._build_fs_set(X, x_i, fs_set)
        
        Y = self._build_embedding(fs_set)
        Y = self._optimize_embedding(fs_set, Y, X)
        return Y
    
    def _calc_distance(self, x, y):
        return distance.cosine(x, y)

    def _build_fs_set(self, X, x_i, fs_set):
        knn, knn_dists = self._find_nearest_neighbors(X, x_i)
        
        p = knn_dists[0]
        sig = self._smooth_knn_dists(knn_dists, p)
        
        for y_i in knn:
            d = max(0, self._calc_distance(X[x_i], X[y_i]) - p) / sig
            fs_set[x_i][y_i] = math.exp(-d)
            
        return fs_set
    
    def _find_nearest_neighbors(self, X, x_i):
        dists = np.array([self._calc_distance(y, X[x_i]) for y in X])
        idx = np.argsort(dists)[1:self.n_neighbors + 1]
        return idx, dists[idx]
    
    def _smooth_knn_dists(self, knn_dists, p):
        sig_lower = 0.0
        sig_upper = 10.0
        
        edge = math.log(self.n_neighbors, 2)
        while (sig_upper - sig_lower > 0.00001):
            sig = (sig_upper + sig_lower) / 2
            w = np.sum(np.exp(-(knn_dists - p) / sig))
            if w > edge:
                sig_upper = sig
            else:
                sig_lower = sig
                
        return sig
    
    def _build_embedding(self, A):
        D = np.diag(np.sum(A > 0, axis=1)) ** (1 / 2)
        B = A + np.transpose(A) -  np.matmul(A, np.transpose(A))
        L = np.matmul(np.matmul(D, (D - B)), D)
    
        w, v = np.linalg.eig(L)
        idx = np.argsort(w)
        evec = v[:,idx]
        
        return np.transpose(evec[1 : self.n_targets + 1])
    
    def _optimize_embedding(self, fs_set, Y, X):
        alpha = 1.0
        for e in tqdm(range(1, self.n_epochs + 1)):
            Y_tmp = copy(Y)

            for x_i in range(len(fs_set)):
                for x_j in range(len(fs_set[x_i])):
                    if random() < fs_set[x_i][x_j]:
                        Y_tmp[x_i] = Y[x_i] + alpha * force_curve(Y[x_i] - Y[x_j],
                                                                  self._calc_distance(X[x_i], X[x_j]),
                                                                  self.a, self.b)

                        neg_inds = np.random.choice(len(Y), 5)
                        for x_k in neg_inds:
                            Y_tmp[x_i] = Y[x_i] + alpha * repilsive_curve(Y[x_i] - Y[x_j],
                                                                          self._calc_distance(X[x_i], X[x_j]),
                                                                          self.a, self.b)
                    
            Y = Y_tmp
            alpha = 1.0 - e / self.n_epochs
                    
        return Y
    

In [596]:
umap = Umap()

1.6083394510691196 0.6296117150454154


In [597]:
images = []
with open('data/fashion/t10k-images-idx3-text') as file:
    for line in file:
        images.append(list(map(float, line.split())))
images = np.array(images)[:1000]
    
labels = []
with open('data/fashion/t10k-labels-idx1-text') as file:
    for line in file:
        labels.append(int(line.strip()))
labels = np.array(labels)[:1000]

In [598]:
len(images), len(labels)

(1000, 1000)

In [ ]:
embedding = umap.fit_transform(images)

 14%|█▍        | 141/1000 [00:19<01:48,  7.90it/s]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors
plt.style.use('seaborn-whitegrid')

In [ ]:
x, y = zip(*embedding)

plt.figure(figsize=(8, 8))
plt.scatter(x, y, c=labels*10, cmap='viridis')
plt.show()

In [560]:
import umap

In [561]:
embedding = umap.UMAP(n_neighbors=5).fit_transform(images)

AttributeError: module 'umap' has no attribute 'UMAP'